## Experimental interactive sketch

#### Imports

In [1]:
from __future__ import print_function # For py 2.7 compat
import ipywidgets as widgets
from traitlets import Unicode
from traitlets import CBytes
from traitlets import CInt
from traitlets import CBool
import numpy as np

#### Python side canvas widget

In [2]:
class P55CanvasWidget(widgets.DOMWidget):
    _view_name = Unicode('P55CanvasView').tag(sync=True)
    _view_module = Unicode('p55canvas').tag(sync=True)
    value = CInt().tag(sync=True)
    a = np.zeros([100,100,4], dtype='uint8') + 128
    sidecar = CBytes(a.tobytes()).tag(sync=True)
    mouse_x = CInt().tag(sync=True)
    mouse_y = CInt().tag(sync=True)
    mouse_down = CBool().tag(sync=True)    

#### Javascript side

In [3]:
%%javascript
requirejs.undef('p55canvas');

define('p55canvas', ["jupyter-js-widgets"], function(widgets) {
    // we track all mouse movement and store into globals
    var p55_mouse_x = 0;
    var p55_mouse_y = 0;
    var p55_mouse_is_down = false;

    var callback_on_mouse_down = function(evt) {
        p55_mouse_is_down = true;
    };
    
    var callback_on_mouse_up = function(evt) {
        p55_mouse_is_down = false;
    };

    function track_mouse_pos(evt) {
        p55_mouse_x = evt.clientX;
        p55_mouse_y = evt.clientY;
    }

    var P55CanvasView = widgets.DOMWidgetView.extend({
        render: function() { 
            var that = this;
            this.$canvas = $('<canvas width="100" height="100"></canvas>');
            this.$el.append(this.$canvas);
            this.value_changed();
            this.model.on('change:value', this.value_changed, this);
            this.$canvas[0].addEventListener('mousemove', track_mouse_pos, false);
            this.$canvas[0].addEventListener('mousedown', callback_on_mouse_down, false);
            this.$canvas[0].addEventListener('mouseup', callback_on_mouse_up, false);
        },
        
        value_changed: function() {
            // anytime the value is changed we do a full sync
            var ctx = this.$canvas[0].getContext("2d");
            var buffer = this.model.get('sidecar').buffer
            var arr = new Uint8ClampedArray(buffer);
            var imdata = new ImageData(arr, 100, 100);
            ctx.putImageData(imdata, 0, 0)
            var rect = this.$canvas[0].getBoundingClientRect();
            var local_x = Math.floor( p55_mouse_x - rect.left );
            var local_y = Math.floor( p55_mouse_y - rect.top );
            this.handle_mouse(local_x, local_y, p55_mouse_is_down);
        },
        
        handle_mouse: function(x, y, is_down) {
            this.model.set('mouse_x', x);
            this.model.set('mouse_y', y);
            this.model.set('mouse_down', is_down);
            this.touch();
        },
        
    });
    
    return {
        P55CanvasView: P55CanvasView
    };
});

<IPython.core.display.Javascript object>

### Try it out

In [4]:
## OK, combine with PIL
from PIL import Image, ImageDraw
import numpy as np
import IPython.display 

In [5]:
widget = P55CanvasWidget()
IPython.display.display(widget)

In [6]:
# draw into above canvas tracking mouse
w, h = 100, 100

widget.value = 0
for n in range(1000):
    # this is a hack to make the widget interactive while loop is processing
    for x in range(100):
        get_ipython().kernel.do_one_iteration()
    x = widget.mouse_x
    y = widget.mouse_y
    is_down = widget.mouse_down
    if is_down:
        img = Image.new('RGBA', [w, h], color=(0,0,255,255))
    else:
        img = Image.new('RGBA', [w, h], color=(255,0,0,255))
    draw = ImageDraw.Draw(img)
    draw.line((0, 0) + (x, y), fill=0)
    widget.sidecar = img.tobytes()
    widget.mouse_x = widget.mouse_x
    widget.mouse_y = widget.mouse_y
    widget.value = widget.value + 1    